In [1]:
pip  install -U farasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import random 
import json
import pickle 
import numpy as np
#import tensorflow
import tensorflow 
from tensorflow import keras
print('tensorflow version', tensorflow.__version__)

tensorflow version 2.8.0


In [14]:
import nltk

In [3]:
from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer
from farasa.diacratizer import FarasaDiacritizer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer

In [4]:
intents = json.loads(open('/content/intents.json', encoding="utf8").read())


In [5]:
stemmer = FarasaStemmer()

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100%|██████████| 241M/241M [00:24<00:00, 9.98MiB/s]


In [21]:
words = []
classes = []
documents = []
ignore_letters = ['!','?','.',',', '؟',')','(','،']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        print("Pattern is in progress")
        word_list = nltk.tokenize.wordpunct_tokenize(pattern)
        #stemmed = stemmer.stem(pattern)
        #words = [stemmer.stem(w) for w in pattern if w not in ignore_letters]
        #words = sorted(list(set(words)))
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag']) 



Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is in progress
Pattern is 

In [22]:
words = [stemmer.stem(w) for w in words if w not in ignore_letters]
#sorted to ignore any duplicates so that the model don't overfit 
words = sorted(list(set(words)))


In [23]:
words

['events',
 'آخر',
 'أجاب',
 'أدنى',
 'أسعار',
 'أعار',
 'أعرف',
 'أقام',
 'أقدر',
 'أقدم',
 'أقرب',
 'أقل',
 'ألعاب',
 'أماكن',
 'أمر',
 'أنواع',
 'أهل',
 'أو',
 'أي',
 'إجراء',
 'إدارة',
 'إعادة',
 'إمكانية',
 'اتقبلت',
 'اثنين',
 'احترازي',
 'اختبار',
 'اد',
 'ازاى',
 'ازاي',
 'استيعاب',
 'اشوف',
 'اصطناعي',
 'اقصد',
 'ال',
 'الى',
 'امتحان',
 'امتحن',
 'امتى',
 'امتي',
 'ان',
 'اوفلاين',
 'اونلاين',
 'اي',
 'ايفينتات',
 'باب',
 'باص',
 'باي',
 'بدأ',
 'بعد',
 'بقي',
 'بنت',
 'بيكون',
 'تأخر',
 'تأخير',
 'تاني',
 'تجمع',
 'تحت',
 'تحديد',
 'تخصص',
 'تخفيض',
 'تدريب',
 'ترحيب',
 'ترم',
 'تسليم',
 'تصحيح',
 'تظلم',
 'تقدم',
 'تقدير',
 'تقديم',
 'تكنولوجيا',
 'تنظيم',
 'توظيف',
 'ثانوي',
 'ثاني',
 'جامع',
 'جامعة',
 'جدول',
 'جدولى',
 'جديد',
 'جيزة',
 'حاجة',
 'حد',
 'حفلة',
 'خاص',
 'خدمة',
 'خر',
 'خط',
 'خطوة',
 'دراسة',
 'درجة',
 'دكتر',
 'دلوقتي',
 'دولى',
 'دي',
 'ديني',
 'ذكاء',
 'ذلك',
 'راح',
 'رحل',
 'رحلة',
 'رسم',
 'رسمي',
 'روبوتس',
 'رياضة',
 'زايد',
 'سابع',
 'سبب',
 'س

In [24]:
labels = sorted(classes)

In [26]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(words, open('classes.pkl','wb'))

In [25]:
training = []
output = []

out_empty = [0 for _ in range (len(labels))]

In [30]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [stemmer.stem(word) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(out_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [31]:
#Last_step in preprocessing 
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [34]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum =0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('SAUL_Chatbot.model')
print("Done")


Epoch 1/200


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


25/25 [==============================] - 1s 2ms/step - loss: 3.6439 - accuracy: 0.0565 
Epoch 2/200
25/25 [==============================] - 0s 2ms/step - loss: 3.5037 - accuracy: 0.1694
Epoch 3/200
25/25 [==============================] - 0s 1ms/step - loss: 3.3128 - accuracy: 0.1694
Epoch 4/200
25/25 [==============================] - 0s 2ms/step - loss: 3.1251 - accuracy: 0.2016
Epoch 5/200
25/25 [==============================] - 0s 1ms/step - loss: 2.9932 - accuracy: 0.2097
Epoch 6/200
25/25 [==============================] - 0s 2ms/step - loss: 2.8876 - accuracy: 0.1935
Epoch 7/200
25/25 [==============================] - 0s 2ms/step - loss: 2.7676 - accuracy: 0.2500
Epoch 8/200
25/25 [==============================] - 0s 2ms/step - loss: 2.6897 - accuracy: 0.2823
Epoch 9/200
25/25 [==============================] - 0s 2ms/step - loss: 2.5935 - accuracy: 0.3145
Epoch 10/200
25/25 [==============================] - 0s 2ms/step - loss: 2.4254 - accuracy: 0.3387
Epoch 11/200
25/25 [